In [3]:
%pip install mediapipe
%pip install opencv-python
%pip install numpy

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [1]:
import mediapipe as mp
mp_pose = mp.solutions.pose
pose = mp_pose.Pose()

In [2]:
import cv2

In [5]:
import cv2
import mediapipe as mp
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
import joblib
# Load the Keras model
model = load_model('exercise_recognition_model2.keras')
label_encoder = joblib.load('label_encoder.joblib')

In [7]:

# Initialize the webcam
cap = cv2.VideoCapture(0)  # '0' is typically the default value for the primary camera

mp_pose = mp.solutions.pose
pose = mp_pose.Pose()

while True:
    ret, frame = cap.read()
    if not ret:
        continue

    # Convert the frame to RGB
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Process the frame with MediaPipe Pose
    results = pose.process(frame_rgb)

    if results.pose_landmarks:
        # Draw the pose annotations on the frame
        mp.solutions.drawing_utils.draw_landmarks(
            frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

        # Extract landmarks
        landmarks = []
        for landmark in results.pose_landmarks.landmark:
            landmarks.extend([landmark.x, landmark.y, landmark.z])

        # Flatten the landmarks into a single array
        landmarks = np.array(landmarks).flatten()

        # Reshape the landmarks to match the expected input shape of the model
        # The expected shape is (1, 1, 99), where num_features should match the number of columns used during training
        landmarks = np.expand_dims(landmarks, axis=0)  # Add the batch dimension
        landmarks = np.expand_dims(landmarks, axis=1)  # Add the extra dimension the model expects

        # Make a prediction
        predictions = model.predict(landmarks)
        pose_id = np.argmax(predictions, axis=1)[0]
        pose_name = label_encoder.inverse_transform([pose_id])[0]  # Decode the pose ID to pose name

        # Display the predicted pose name or id on the frame
        cv2.putText(frame, f'Pose: {pose_name}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
        # cv2.putText(frame, f'Pose: {pose_name}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)

    cv2.imshow('Pose Detection', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the webcam and destroy all OpenCV windows
cap.release()
cv2.destroyAllWindows()

1/1 [==============================] - 0s 103ms/step


KeyboardInterrupt: 